In [1]:
%matplotlib inline

import glob
import os
from datetime import datetime

import pandas as pd
import numpy as np
from lxml import etree
import matplotlib.pyplot as plt
import seaborn as sns
import pyprind

import qgrid

from diligent import diligent

pd.set_option('display.max_columns', 400)


def progress_pandas_df():
    def inner(df, func, *args, **kwargs):
        df_len = len(df) + 1
        progress_logger = pyprind.ProgPercent(df_len)
        def wrapper(*args, **kwargs):
            progress_logger.update()
            return func(*args, **kwargs)
        result = df.apply(wrapper, *args, **kwargs)
        return result
    pd.DataFrame.progress_apply = inner

progress_pandas_df()

In [2]:
qgrid.nbinstall()

In [3]:
filenames = glob.glob('data/datenexport_20160407/*.xml')

In [4]:
import itertools

XML_INFO = {
    'traeger_art': 'Traeger/Art/text()',
    'traeger_name': 'Traeger/Name/text()',
    'traeger_verband': 'Traeger/Verband_Der_Pflegeeinrichtung/text()',
    'start_date': 'Datum_Inbetriebnahme/text()',
    'start_date_contract': 'Datum_Versorgungsvertrag/text()',
    'belegt_vollstationär': 'Belegte_Plaetze/Vollstationaere_Pflege/text()',
    'belegt_kurzzeitpflege': 'Belegte_Plaetze/Kurzzeitpflege/text()',
    'personal_total': 'Personal/Gesamtzahl/text()',
    'personal_pflege': 'Personal/Pflege_Betreuung/text()',
    'grade_overall': 'Ergebnis[@bereich = "Gesamtnote"]/@note',
    'grade_care': 'Ergebnis[@bereich = "Bereich_1"]/@note',
    'grade_dementia': 'Ergebnis[@bereich = "Bereich_2"]/@note',
    'grade_social': 'Ergebnis[@bereich = "Bereich_3"]/@note',
    'grade_housing': 'Ergebnis[@bereich = "Bereich_4"]/@note',
    'grade_residents': 'Ergebnis[@bereich = "Bereich_5"]/@note',
    'check_type': 'Pruefungsart/text()',
    'insurance': 'Zustaendiger_LV_der_Pflegekassen/text()',
}

def get_report_vars():
    for i in itertools.chain(range(1, 32), range(33, 37), [40]):
        yield 'T' + str(i).zfill(2)

def get_boolean_vars():
    for i in itertools.chain([32], range(37, 40), range(41, 60)):
        yield 'T' + str(i).zfill(2)

def get_questionaire_vars():
    for i in range(60, 78):
        yield 'T' + str(i).zfill(2)

for k in get_report_vars():
    XML_INFO[k] = 'Kriterium_ErfuelltBeiVon[@kriterium = "%s"]/@anzahl_bei' % k
    XML_INFO['%s_total' % k] = 'Kriterium_ErfuelltBeiVon[@kriterium = "%s"]/@anzahl_von' % k
                              
for k in get_boolean_vars():
    XML_INFO[k] = 'Kriterium_JaNein[@kriterium = "%s"]/@ergebnis' % k
    
questionaire_cats = ['always', 'often', 'sometimes', 'never']

for k in get_questionaire_vars():
    XML_INFO[k + '_always'] = 'Kriterium_ErfuelltBeiVonQB5[@kriterium = "%s"]/@anzahl_bei_immer' % k
    XML_INFO[k + '_often'] = 'Kriterium_ErfuelltBeiVonQB5[@kriterium = "%s"]/@anzahl_bei_haeufig' % k
    XML_INFO[k + '_sometimes'] = 'Kriterium_ErfuelltBeiVonQB5[@kriterium = "%s"]/@anzahl_bei_gelegentlich' % k
    XML_INFO[k + '_never'] = 'Kriterium_ErfuelltBeiVonQB5[@kriterium = "%s"]/@anzahl_bei_nie' % k
    

def fix_xpath(xp):
    parts = xp.split('/')
    new_parts = []
    for p in parts:
        if p[0] != p[0].lower():
            p = '{ns}' + p
        new_parts.append(p)
    return '//' + '/'.join(new_parts)
        

NAMESPACE = 'GI4X:/xml-schema/Pflegebericht/1.0.13'

def xpath(root, xpath, ns=NAMESPACE):
    return etree.ETXPath(xpath.format(ns='{%s}' % ns))(root)


def get_first(iterable, default=None):
    if iterable:
        for item in iterable:
            return str(item)
    return default


def get_xml_data(tid):
    filename = 'data/datenexport_20160407/transparenzbericht_%d.xml' % tid
    if not os.path.exists(filename):
        return {
            k: None for k, _ in XML_INFO.items()
        }
    root = etree.fromstring(open(filename, 'rb').read())
    return {
        k: get_first(xpath(root, fix_xpath(v))) for k, v in XML_INFO.items()
    }

def add_xml_info(row):
    d = get_xml_data(row['ID'])
    for k, v in d.items():
        row[k] = v
    return row

def make_int(series):
    return pd.to_numeric(series, errors='coerce')


def apply_xml(df):
    df = df.progress_apply(add_xml_info, axis=1)
    for c in ('grade_overall', 'grade_care'):
        df[c] = make_int(df[c])
    for c in ('start_date', 'start_date_contract'):
        df[c] = pd.to_datetime(df[c])
    for k in get_report_vars():
        df[k] = make_int(df[k])
        df[k + '_total'] = make_int(df[k + '_total'])
    for k in get_boolean_vars():
        df[k] = df[k] == 'J'
    for k in get_questionaire_vars():
        for cat in questionaire_cats:
            key = '%s_%s' % (k, cat)
            df[key] = make_int(df[key])
        
    return df

In [5]:
import json
from rtree import index

from shapely.geometry import shape, Point

class GeoIndex(object):
    def __init__(self, filename):
        with open(filename) as f:
            js = json.load(f)
        self.idx = index.Index()
        self.features = js['features']
    
        for i, feature in enumerate(self.features):
            polygon = shape(feature['geometry'])
            self.idx.insert(i, polygon.bounds)

    def covers(self, lat, lng):
        point = Point(lng, lat)
        for j in self.idx.intersection(point.coords[0]):
            polygon = shape(self.features[j]['geometry'])
            if point.within(polygon):
                yield self.features[j]['properties']
    
    def get_feature_property(self, lat, lng, prop):
        l = list(self.covers(lat, lng))
        if l:
            return l[0][prop]
        return None
    
laender_index = GeoIndex('geodata/bundeslaender.geojson')
kreise_index = GeoIndex('geodata/landkreise.geojson')
assign_geofeature = lambda idx, key: lambda x: idx.get_feature_property(x.lat, x.lng, key)

In [6]:
RED_FLAGS = {
    'red_flag_decubitus': [1, 2, 5, 6],
    'red_flag_food': [7, 8, 9, 11],
    'red_flag_pain': [13, 14, 15],
    'red_flag_incontinence': [16, 17],
    'red_flag_medicine': [23, 24, 25, 26]
}

def get_red_flag(df, key):
    '''
    „Dekubitus“ (question 1, 2, 5, 6)
    „Nahrungsversorung“ (question 7, 8, 9, 11)
    „Schmerzen“ (question 13, 14, 15)
    „Inkontinenz“ (question 16, 17)
    „Medikamente“ (23, 24, 25, 26)
    '''

    return sum(df['T%s_total' % str(x).zfill(2)] - df['T%s' % str(x).zfill(2)] for x in RED_FLAGS[key]) > 0

In [143]:
def read_nursinghomes(filename):
    df = pd.read_csv(filename, parse_dates=['start_date', 'start_date_contract', 'letzte Aktualisierung durch Pflegeeinrichtung'], encoding='utf-8')
    df['ags'] = df['ags'].apply(lambda x: None if pd.isnull(x) else str(int(x)).zfill(5))
    df['PLZ'] = df['PLZ'].apply(lambda x: None if pd.isnull(x) else str(int(x)).zfill(5))
    return df


def get_nursinghome_csv(filename='data/nursinghomes.csv'):
    if os.path.exists(filename):
        return read_nursinghomes(filename)

    print('Read in CSV')
    df = pd.read_csv('data/datenexport_20160407/correctivExport.csv', encoding='latin1', sep=';')

    split_indicator = ['Minimum/Maximum', 'Gesamtpreis/Anteil Pflegekasse/Eigenanteil', 'Gesamtpreis/Anteil Pflegekasse']

    cost_columns = [c for c in df.columns if any([s in c for s in split_indicator])]

    def split_cost_column(column):
        def _split_cost_column(row):
            if pd.isnull(row[column]):
                return row
            amounts = row[column].replace('.', '').replace(',', '.').split('/')
            parts = column.split('/')
            name, first_part = parts[0].rsplit(' ', 1)
            name_parts = [first_part] + parts[1:]
            for n, a in zip(name_parts, amounts):
                row['%s %s' % (name, n)] = float(a) if a != 'null' else None
            return row
        return _split_cost_column

    print('Split column values')
    for cost_column in cost_columns:
        df = df.progress_apply(split_cost_column(cost_column), axis=1)
        del df[cost_column]

    print('Applying Geo Features')

    df = df.rename(columns={
        'Koordinate Latitude': 'lat',
        'Koordinate Longitude': 'lng'
    })
    
    df = df.drop_duplicates(['Straße Hausnr', 'PLZ', 'lat', 'lng'], keep='first')

    print('Apply bundesland, landkreis, AGS')
    df['bundesland'] = df[df.lat.notnull()].apply(assign_geofeature(laender_index, 'GEN'), axis=1)
    df['landkreis'] = df[df.lat.notnull()].apply(assign_geofeature(kreise_index, 'GEN'), axis=1)
    df['ags'] = df[df.lat.notnull()].apply(assign_geofeature(kreise_index, 'RS'), axis=1)
    ags_fixes = {
        'DRK Seniorenhaus "Bernstein"': '14521',
        'Itertalklinik Seniorenzentrum Roetgen': '05334',
        'VITOS Heilpädagogische Einrichtung Riedstadt': '06433',
        'Haus Mylius Lucia Mylius Altenpfegeheim': '08126',
        'Seniorenzentrum Efringen-Kirchen': '08336',
        'Seniorenzentrum Efringen-Kirchen Tagespflege': '08336',
        'Pflegeabteilung St. Gabriel': '09162',
    }

    for name, ags in ags_fixes.items():
        df.loc[df.ags.isnull() & (df.Name == name), 'ags'] = ags

    ags_mapping = pd.read_csv('data/ags_county_state.csv', dtype='O')
    df.loc[df['bundesland'].isnull(), 'bundesland'] = df.loc[df['bundesland'].isnull(), 'ags'].map(lambda x: ags_mapping[ags_mapping['ags'] == x]['state'].iloc[0])
    df.loc[df['landkreis'].isnull(), 'landkreis'] = df.loc[df['landkreis'].isnull(), 'ags'].map(lambda x: ags_mapping[ags_mapping['ags'] == x]['name'].iloc[0])
    
    df['Web'] = df['Web'].str.replace(' ', '').str.lower()
    
    df = df.merge(pd.read_csv('data/aok_mapping.csv', dtype=int), how='left', on='ID')
    df = df.merge(pd.read_csv('data/aok_transparency_mapping.csv', dtype=int), how='left', on='aok_id')

    care_types = {
        'tagespflege': 'Tagespflege',
        'nachtpflege': 'Nachtpflege',
        'kurzzeitpflege': 'Kurzzeitpflege',
        'vollstationaer': 'Vollstationär'
    }
    columns = list(df.columns)
    care_columns = {k: [x for x in columns if x.startswith(v)] for k, v in care_types.items()}

    print('Adding Care type flags')
    for k in care_types:
        df[k] = df[care_columns[k]].any(1)


    print('Applying XML')
    df = apply_xml(df)
    
    print('Adding red flags')
    for key in RED_FLAGS:
        df[key] = get_red_flag(df, key)
    
    print('Exporting...')
    df.to_csv(filename, encoding='utf-8', index=False)
    return df


df = get_nursinghome_csv()

df.head()

,Ausstattung/Angebote: Bibliothek,Ausstattung/Angebote: Eigene Möbel,Ausstattung/Angebote: Fahrdienst,Ausstattung/Angebote: Fernseher,Ausstattung/Angebote: Garten,Ausstattung/Angebote: Hallenbad,Ausstattung/Angebote: Haustiere,Ausstattung/Angebote: Internet,Ausstattung/Angebote: Kegelbahn,Ausstattung/Angebote: Probewohnen,Ausstattung/Angebote: Sauna,Ausstattung/Angebote: Telefon,Ausstattung/Angebote: Übernachtung von Angehörigen,E-Mail,Fax,ID,Kontakt,lat,lng,Kurzzeitpflege Allgemein Härtefall Anteil Pflegekasse,Kurzzeitpflege Allgemein Härtefall Eigenanteil,Kurzzeitpflege Allgemein Härtefall Gesamtpreis,Kurzzeitpflege Allgemein Investitionskosten Maximum,Kurzzeitpflege Allgemein Investitionskosten Minimum,Kurzzeitpflege Allgemein Pflegestufe 1 Anteil Pflegekasse,Kurzzeitpflege Allgemein Pflegestufe 1 Eigenanteil,Kurzzeitpflege Allgemein Pflegestufe 1 Gesamtpreis,Kurzzeitpflege Allgemein Pflegestufe 2 Anteil Pflegekasse,Kurzzeitpflege Allgemein Pflegestufe 2 Eigenanteil,Kurzzeitpflege Allgemein Pflegestufe 2 Gesamtpreis,Kurzzeitpflege Allgemein Pflegestufe 3 Anteil Pflegekasse,Kurzzeitpflege Allgemein Pflegestufe 3 Eigenanteil,Kurzzeitpflege Allgemein Pflegestufe 3 Gesamtpreis,Kurzzeitpflege Beatmungspflichtig Härtefall Anteil Pflegekasse,Kurzzeitpflege Beatmungspflichtig Härtefall Eigenanteil,Kurzzeitpflege Beatmungspflichtig Härtefall Gesamtpreis,Kurzzeitpflege Beatmungspflichtig Investitionskosten Maximum,Kurzzeitpflege Beatmungspflichtig Investitionskosten Minimum,Kurzzeitpflege Beatmungspflichtig Pflegestufe 1 Anteil Pflegekasse,Kurzzeitpflege Beatmungspflichtig Pflegestufe 1 Eigenanteil,Kurzzeitpflege Beatmungspflichtig Pflegestufe 1 Gesamtpreis,Kurzzeitpflege Beatmungspflichtig Pflegestufe 2 Anteil Pflegekasse,Kurzzeitpflege Beatmungspflichtig Pflegestufe 2 Eigenanteil,Kurzzeitpflege Beatmungspflichtig Pflegestufe 2 Gesamtpreis,Kurzzeitpflege Beatmungspflichtig Pflegestufe 3 Anteil Pflegekasse,Kurzzeitpflege Beatmungspflichtig Pflegestufe 3 Eigenanteil,Kurzzeitpflege Beatmungspflichtig Pflegestufe 3 Gesamtpreis,Kurzzeitpflege Demenzkrank Härtefall Anteil Pflegekasse,Kurzzeitpflege Demenzkrank Härtefall Eigenanteil,Kurzzeitpflege Demenzkrank Härtefall Gesamtpreis,Kurzzeitpflege Demenzkrank Investitionskosten Maximum,Kurzzeitpflege Demenzkrank Investitionskosten Minimum,Kurzzeitpflege Demenzkrank Pflegestufe 1 Anteil Pflegekasse,Kurzzeitpflege Demenzkrank Pflegestufe 1 Eigenanteil,Kurzzeitpflege Demenzkrank Pflegestufe 1 Gesamtpreis,Kurzzeitpflege Demenzkrank Pflegestufe 2 Anteil Pflegekasse,Kurzzeitpflege Demenzkrank Pflegestufe 2 Eigenanteil,Kurzzeitpflege Demenzkrank Pflegestufe 2 Gesamtpreis,Kurzzeitpflege Demenzkrank Pflegestufe 3 Anteil Pflegekasse,Kurzzeitpflege Demenzkrank Pflegestufe 3 Eigenanteil,Kurzzeitpflege Demenzkrank Pflegestufe 3 Gesamtpreis,Kurzzeitpflege Wachkoma Härtefall Anteil Pflegekasse,Kurzzeitpflege Wachkoma Härtefall Eigenanteil,Kurzzeitpflege Wachkoma Härtefall Gesamtpreis,Kurzzeitpflege Wachkoma Investitionskosten Maximum,Kurzzeitpflege Wachkoma Investitionskosten Minimum,Kurzzeitpflege Wachkoma Pflegestufe 1 Anteil Pflegekasse,Kurzzeitpflege Wachkoma Pflegestufe 1 Eigenanteil,Kurzzeitpflege Wachkoma Pflegestufe 1 Gesamtpreis,Kurzzeitpflege Wachkoma Pflegestufe 2 Anteil Pflegekasse,Kurzzeitpflege Wachkoma Pflegestufe 2 Eigenanteil,Kurzzeitpflege Wachkoma Pflegestufe 2 Gesamtpreis,Kurzzeitpflege Wachkoma Pflegestufe 3 Anteil Pflegekasse,Kurzzeitpflege Wachkoma Pflegestufe 3 Eigenanteil,Kurzzeitpflege Wachkoma Pflegestufe 3 Gesamtpreis,Nachtpflege Allgemein Investitionskosten Maximum,Nachtpflege Allgemein Investitionskosten Minimum,Nachtpflege Allgemein Pflegestufe 1 Anteil Pflegekasse,Nachtpflege Allgemein Pflegestufe 1 Eigenanteil,Nachtpflege Allgemein Pflegestufe 1 Gesamtpreis,Nachtpflege Allgemein Pflegestufe 2 Anteil Pflegekasse,Nachtpflege Allgemein Pflegestufe 2 Eigenanteil,Nachtpflege Allgemein Pflegestufe 2 Gesamtpreis,Nachtpflege Allgemein Pflegestufe 3 Anteil Pflegekasse,Nachtpflege Al

In [8]:
df['insurance'].value_counts(dropna=False)

AOK     5993
NaN     3822
vdek    2880
IKK      886
BKK      742
KBS      494
LKK      118
Name: insurance, dtype: int64

In [173]:
df_fs = df[df.vollstationaer]

len(df_fs), len(df_fs[df_fs.duplicated(['Straße Hausnr', 'PLZ', 'lat', 'lng'], keep='last')])

(11408, 410)

## What’s the county with the most expensive nursing homes? (Pflegeklasse 1, 2, 3 + UV / jeweils für Dauerpflege und Kurzzeitpflege)

In [10]:
def get_top_10_by_value(df, value, index='landkreis', top=10, asc=False):
    print('## ' + value)
    print(df.pivot_table(index=index, values=value, aggfunc=np.mean).sort_values(ascending=asc).head(10))
    print('-' * 20)

columns = [c for c in df.columns if 'Vollstationär Allgemein' in c and 'Gesamtpreis' in c]
for c in columns:
    get_top_10_by_value(df, c)

## Vollstationär Allgemein Härtefall Gesamtpreis
landkreis
Stuttgart                      4186.301471
St. Wendel                     4152.600000
Böblingen                      4128.381081
Steinfurt                      4090.200000
Landau in der Pfalz            4061.500000
Eifelkreis Bitburg-Prüm        4054.800000
Trier                          4053.200000
Neunkirchen                    4030.617391
Regionalverband Saarbrücken    4026.273333
Ludwigsburg                    4023.319565
Name: Vollstationär Allgemein Härtefall Gesamtpreis, dtype: float64
--------------------
## Vollstationär Allgemein Pflegestufe 1 Gesamtpreis
landkreis
Siegen-Wittgenstein    2788.850000
Kaufbeuren             2716.140000
Köln                   2705.416484
Krefeld                2653.834615
Remscheid              2632.328571
München                2630.439759
St. Wendel             2614.600000
Mülheim an der Ruhr    2608.263158
Steinfurt              2598.656897
Viersen                2597.100000
Name: Vol

In [11]:
columns = [c for c in df.columns if 'Kurzzeitpflege Allgemein' in c and 'Gesamtpreis' in c]
for c in columns:
    get_top_10_by_value(df, c)

## Kurzzeitpflege Allgemein Härtefall Gesamtpreis
landkreis
Cloppenburg                    4758.600000
Neckar-Odenwald-Kreis          4746.300000
Osnabrück                      4493.700000
Reutlingen                     4398.525000
Sigmaringen                    4219.500000
Würzburg                       4173.900000
Neunkirchen                    4171.125000
Stuttgart                      4104.750000
Regionalverband Saarbrücken    4082.817391
Rems-Murr-Kreis                4043.127273
Name: Kurzzeitpflege Allgemein Härtefall Gesamtpreis, dtype: float64
--------------------
## Kurzzeitpflege Allgemein Pflegestufe 1 Gesamtpreis
landkreis
Neckar-Odenwald-Kreis    4746.300000
Heidelberg               3496.800000
Würzburg                 3060.900000
München                  2952.900000
Fürstenfeldbruck         2910.600000
Berchtesgadener Land     2845.200000
Berlin                   2766.763636
Ravensburg               2741.340000
Reutlingen               2731.140000
Sigmaringen            

## What’s the county with the cheapest nursing homes? (Pflegeklasse 1, 2, 3 + UV / jeweils für Dauerpflege und Kurzzeitpflege)

In [12]:
columns = [c for c in df.columns if 'Vollstationär Allgemein' in c and 'Gesamtpreis' in c]
for c in columns:
    get_top_10_by_value(df, c, asc=True)

## Vollstationär Allgemein Härtefall Gesamtpreis
landkreis
Wilhelmshaven            2727.521053
Unstrut-Hainich-Kreis    2825.385000
Osterode am Harz         2860.848387
Uelzen                   2861.466667
Holzminden               2866.248000
Peine                    2871.912500
Rostock                  2872.730769
Schaumburg               2875.104878
Lüneburg                 2876.381250
Ludwigslust-Parchim      2881.008333
Name: Vollstationär Allgemein Härtefall Gesamtpreis, dtype: float64
--------------------
## Vollstationär Allgemein Pflegestufe 1 Gesamtpreis
landkreis
Unstrut-Hainich-Kreis    1619.580000
Dresden                  1623.621429
Chemnitz                 1641.193548
Mittelsachsen            1648.406897
Leipzig                  1651.989320
Rostock                  1655.511864
Ludwigslust-Parchim      1657.708333
Mansfeld-Südharz         1658.545455
Zwickau                  1660.189655
Vogtlandkreis            1664.175000
Name: Vollstationär Allgemein Pflegestufe 1 Gesam

In [13]:
columns = [c for c in df.columns if 'Kurzzeitpflege Allgemein' in c and 'Gesamtpreis' in c]
for c in columns:
    get_top_10_by_value(df, c, asc=True)

## Kurzzeitpflege Allgemein Härtefall Gesamtpreis
landkreis
Mannheim                 2136.600000
Herzogtum Lauenburg      2545.053659
Rendsburg-Eckernförde    2553.117647
Steinburg                2587.977778
Oldenburg                2592.300000
Segeberg                 2596.535714
Ostholstein              2605.654286
Schleswig-Flensburg      2659.555102
Nordfriesland            2670.171429
Dithmarschen             2678.334375
Name: Kurzzeitpflege Allgemein Härtefall Gesamtpreis, dtype: float64
--------------------
## Kurzzeitpflege Allgemein Pflegestufe 1 Gesamtpreis
landkreis
Oldenburg                1567.500000
Unstrut-Hainich-Kreis    1619.580000
Vogtlandkreis            1653.042857
Friesland                1661.400000
Nordwestmecklenburg      1673.130000
Saalfeld-Rudolstadt      1685.333333
Gotha                    1685.966667
Kyffhäuserkreis          1688.841176
Greiz                    1691.175000
Wittmund                 1694.100000
Name: Kurzzeitpflege Allgemein Pflegestufe 1 G

### These questions related to cost can also be calculated compared to the the gross domestic product of the counties / states. It can also be calculated compared to the annual income of the people living in these counties / states.

In [14]:
def get_income_df(sheet='PEK je Einwohner'):
    income_df = pd.read_excel('data/VGR_KreisergebnisseBand3_5820008147005.xlsx', sheetname=sheet, skiprows=4)
    income_df = income_df.rename(columns={
            'Regional-schlüssel': 'ags'
        })
    income_df = income_df[~income_df['ags'].isnull()]
    income_df['ags'] = income_df['ags'].apply(lambda x: str(int(x)).ljust(5, '0'))

    income_years = list(range(2000, 2014))
    id_vars = ['ags', 'Gebietseinheit']
    income_df = pd.melt(income_df[income_years + id_vars], id_vars=id_vars, var_name='year', value_name=sheet)
    income_df[sheet] = pd.to_numeric(income_df[sheet], errors='coerce')
    income_df = income_df.set_index('ags')
    return income_df

In [15]:
income_df = get_income_df()
income_df[income_df.year == 2013].sort_values(by='PEK je Einwohner').head()

,Gebietseinheit,year,PEK je Einwohner
ags,,,
15087,"Mansfeld-Südharz, Landkreis",2013,15278.0
14626,"Görlitz, Landkreis",2013,15390.0
40120,"Bremerhaven, Kreisfreie Stadt",2013,15467.0
15089,Salzlandkreis,2013,15846.0
12066,"Oberspreewald-Lausitz, Landkreis",2013,15924.0


In [16]:
income_df[income_df['Gebietseinheit'] == 'Berlin']

,Gebietseinheit,year,PEK je Einwohner
ags,,,
11000,Berlin,2000,17153.0
11000,Berlin,2001,17146.0
11000,Berlin,2002,16881.0
11000,Berlin,2003,16782.0
11000,Berlin,2004,16850.0
11000,Berlin,2005,16811.0
11000,Berlin,2006,17285.0
11000,Berlin,2007,17712.0
11000,Berlin,2008,18313.0


In [17]:
if 'PEK je Einwohner' not in df:
    df = df.merge(income_df[income_df.year == 2013][['PEK je Einwohner']], left_on='ags', right_index=True, how='left')


In [18]:
columns = [c for c in df.columns if 'Vollstationär Allgemein' in c and 'Gesamtpreis' in c]
for c in columns:
    df['%s Ratio PEK/Einwohner' % c] = df[c] / df['PEK je Einwohner']

In [19]:
columns = [c for c in df.columns if 'Vollstationär Allgemein' in c and 'Gesamtpreis' in c and 'Ratio PEK/Einwohner' in c]
for c in columns:
    get_top_10_by_value(df, c)

## Vollstationär Allgemein Härtefall Gesamtpreis Ratio PEK/Einwohner
landkreis
Görlitz                        0.201192
Uckermark                      0.195260
Merzig-Wadern                  0.194325
Nordhausen                     0.194079
Regionalverband Saarbrücken    0.192075
Oberspreewald-Lausitz          0.191347
Neunkirchen                    0.191079
Elbe-Elster                    0.190921
Altenburger Land               0.190570
Vorpommern-Greifswald          0.190160
Name: Vollstationär Allgemein Härtefall Gesamtpreis Ratio PEK/Einwohner, dtype: float64
--------------------
## Vollstationär Allgemein Pflegestufe 1 Gesamtpreis Ratio PEK/Einwohner
landkreis
Uckermark                      0.119633
Merzig-Wadern                  0.117259
Regionalverband Saarbrücken    0.116738
Elbe-Elster                    0.116148
Oberspreewald-Lausitz          0.115960
Neunkirchen                    0.115555
Görlitz                        0.113315
Frankfurt (Oder)               0.112766
Nordhause

In [20]:
income_df.sort_values('PEK je Einwohner').head()

,Gebietseinheit,year,PEK je Einwohner
ags,,,
15087,"Mansfeld-Südharz, Landkreis",2000,10704.0
15089,Salzlandkreis,2000,10848.0
14626,"Görlitz, Landkreis",2000,10884.0
15082,"Anhalt-Bitterfeld, Landkreis",2000,10895.0
12066,"Oberspreewald-Lausitz, Landkreis",2000,10963.0


## How old are the nursing homes on a national / state / county level?

In [21]:
df['days_in_operation'] = (datetime.now() - df['start_date']).apply(lambda x: None if pd.isnull(x) else x.days)
print('Median years overall', df['days_in_operation'].median() / 365)
df.groupby('bundesland')['days_in_operation'].median() / 365

Median years overall 16.263013698630136


bundesland
Baden-Württemberg         15.461644
Bayern                    18.846575
Berlin                    13.928767
Brandenburg               12.980822
Bremen                    17.098630
Hamburg                   27.054795
Hessen                    16.931507
Mecklenburg-Vorpommern    15.504110
Niedersachsen             15.079452
Nordrhein-Westfalen       19.846575
Rheinland-Pfalz           18.520548
Saarland                  11.175342
Sachsen                   14.216438
Sachsen-Anhalt            14.010959
Schleswig-Holstein        16.427397
Thüringen                 13.339726
Name: days_in_operation, dtype: float64

The top and bottom 10 of counties by median age of their nursing homes, and the number of nursing homes in that county.

In [22]:
def median_years(x):
    return x.median() / 365

median_age_years_counties = df.groupby(['ags', 'landkreis'])['days_in_operation'].agg([median_years, len]).sort_values(by='median_years')
median_age_years_counties.head(10)

,,median_years,len
ags,landkreis,,
07340,Südwestpfalz,6.753425,9.0
10043,Neunkirchen,6.984932,26.0
03458,Oldenburg,7.819178,44.0
09679,Würzburg,8.090411,29.0
16074,Saale-Holzland-Kreis,8.252055,15.0
16069,Hildburghausen,8.401370,19.0
16056,Eisenach,9.134247,13.0
16054,Suhl,9.304110,12.0
10046,St. Wendel,9.341096,23.0


In [23]:
median_age_years_counties.tail(10)

,,median_years,len
ags,landkreis,,
09261,Landshut,30.749315,11.0
05316,Leverkusen,30.769863,16.0
07320,Zweibrücken,31.715068,3.0
09463,Coburg,32.520548,10.0
09764,Memmingen,32.689041,7.0
07313,Landau in der Pfalz,33.024658,5.0
09461,Bamberg,35.817808,14.0
09163,Rosenheim,40.605479,5.0
07111,Koblenz,43.632877,17.0


### Oldest and youngest nursing homes in data

In [24]:
print('%d of %d nursing homes without a start date' % (df['start_date'].isnull().sum(), len(df)))

4122 of 14935 nursing homes without a start date


In [25]:
df[['Name', 'PLZ', 'days_in_operation', 'start_date']].sort_values(by='days_in_operation', ascending=False).head()

,Name,PLZ,days_in_operation,start_date
13441,Fürstlich Waldburg-Zeil'sche Hospitalverwaltung,88353,79044.0,1800-01-01
11298,Stiftung ev. Altenheim Ludwigsburg,71640,65683.0,1836-08-01
12190,Alten- und Pflegeheim Braunsche Stiftung,76761,62243.0,1846-01-01
13498,Spital- und Spendfonds Überlingen St. Franzisk...,88662,59686.0,1853-01-01
8952,Schenkel-Schoeller-Stift,52355,57120.0,1860-01-11


In [26]:
df[['Name', 'PLZ', 'days_in_operation', 'start_date']][df['days_in_operation'].notnull()].sort_values(by='days_in_operation', ascending=False).tail()

,Name,PLZ,days_in_operation,start_date
1041,Altenpflegeheim Luisenhaus,06618,219.0,2015-10-26
4627,Seniorenbetreuung Neuenkirchen,28790,213.0,2015-11-01
5891,Seniorenheim Landgrafenblick GmbH,35039,213.0,2015-11-01
3559,Domicil-Seniorenpflegeheim Kirchhofallee GmbH ...,24114,213.0,2015-11-01
1960,"Hauptstadtpflege, Haus Sommerstraße - Kurzzeit...",13409,152.0,2016-01-01


## Where are the best nursing homes according to the grades for 1) overall and 2) medical and nursing?

In [27]:
print('Median overall grade', df['grade_overall'].median())
df.groupby('bundesland')['grade_overall'].median()

Median overall grade 1.2


bundesland
Baden-Württemberg         1.1
Bayern                    1.3
Berlin                    1.1
Brandenburg               1.1
Bremen                    1.3
Hamburg                   1.3
Hessen                    1.2
Mecklenburg-Vorpommern    1.1
Niedersachsen             1.2
Nordrhein-Westfalen       1.2
Rheinland-Pfalz           1.4
Saarland                  1.0
Sachsen                   1.1
Sachsen-Anhalt            1.1
Schleswig-Holstein        1.3
Thüringen                 1.3
Name: grade_overall, dtype: float64

Showing the bottom counties, top counties are all 1.0

In [28]:
median_grade_counties = df.groupby(['ags', 'landkreis'])['grade_overall'].agg([np.median, len]).sort_values(by='median')
median_grade_counties.tail(10)

,,median,len
ags,landkreis,,
09173,Bad Tölz-Wolfratshausen,1.55,14.0
07137,Mayen-Koblenz,1.55,28.0
05914,Hagen,1.55,28.0
07332,Bad Dürkheim,1.55,15.0
07135,Cochem-Zell,1.60,14.0
09463,Coburg,1.60,10.0
09571,Ansbach,1.60,29.0
09577,Weißenburg-Gunzenhausen,1.65,17.0
07141,Rhein-Lahn-Kreis,1.70,15.0


In [29]:
median_grade_counties.sort_values('median').head(10)

,,median,len
ags,landkreis,,
10046,St. Wendel,1.0,23.0
15091,Wittenberg,1.0,21.0
05334,Städteregion Aachen,1.0,106.0
06632,Hersfeld-Rotenburg,1.0,24.0
12062,Elbe-Elster,1.0,25.0
05370,Heinsberg,1.0,68.0
05158,Mettmann,1.0,78.0
05122,Solingen,1.0,39.0
05120,Remscheid,1.0,24.0


### Medical grade

In [30]:
print('Median overall grade', df['grade_care'].median())
df.groupby('bundesland')['grade_care'].median()

Median overall grade 1.4


bundesland
Baden-Württemberg         1.30
Bayern                    1.50
Berlin                    1.20
Brandenburg               1.20
Bremen                    1.60
Hamburg                   1.60
Hessen                    1.30
Mecklenburg-Vorpommern    1.30
Niedersachsen             1.50
Nordrhein-Westfalen       1.30
Rheinland-Pfalz           1.70
Saarland                  1.10
Sachsen                   1.20
Sachsen-Anhalt            1.30
Schleswig-Holstein        1.50
Thüringen                 1.55
Name: grade_care, dtype: float64

In [31]:
median_grade_care_counties = df.groupby(['ags', 'landkreis'])['grade_care'].agg([np.median, len]).sort_values(by='median')
median_grade_care_counties.tail(10)

,,median,len
ags,landkreis,,
07332,Bad Dürkheim,1.95,15.0
07138,Neuwied,1.95,26.0
16068,Sömmerda,2.00,14.0
07334,Germersheim,2.00,13.0
07137,Mayen-Koblenz,2.00,28.0
09571,Ansbach,2.10,29.0
09173,Bad Tölz-Wolfratshausen,2.10,14.0
07141,Rhein-Lahn-Kreis,2.30,15.0
07135,Cochem-Zell,2.30,14.0


In [32]:
median_grade_care_counties.sort_values('median').head(10)

,,median,len
ags,landkreis,,
10046,St. Wendel,1.0,23.0
12070,Prignitz,1.0,26.0
12072,Teltow-Fläming,1.0,30.0
12073,Uckermark,1.0,30.0
05370,Heinsberg,1.0,68.0
05334,Städteregion Aachen,1.1,106.0
10042,Merzig-Wadern,1.1,24.0
12063,Havelland,1.1,30.0
09180,Garmisch-Partenkirchen,1.1,17.0


### How many nursing homes have which kind of owners (private vs non-profit) on a national / state / county level?

In [33]:
print('{} nursing homes without ownership kind'.format(df['traeger_art'].isnull().sum()))
df['traeger_art'].value_counts()

3822 nursing homes without ownership kind


freigemeinnützig      6135
privat                4509
öffentlich             432
nicht zu ermitteln      37
Name: traeger_art, dtype: int64

In [34]:
# In percent
df['traeger_art'].value_counts() / df['traeger_art'].value_counts().sum() * 100

freigemeinnützig      55.205615
privat                40.574102
öffentlich             3.887339
nicht zu ermitteln     0.332943
Name: traeger_art, dtype: float64

In [35]:
ownership_kind = df.groupby(['bundesland', 'traeger_art']).size()

ownership_kind_pivot = ownership_kind.reset_index().pivot_table(index='bundesland', columns='traeger_art',
                                         values=0, fill_value=0, margins=True)
ownership_kind_pivot

traeger_art,freigemeinnützig,nicht zu ermitteln,privat,öffentlich,All
bundesland,,,,,
Baden-Württemberg,929.0000,0.000000,442.0000,100.000000,490.333333
Bayern,860.0000,0.000000,510.0000,140.000000,503.333333
Berlin,129.0000,1.000000,168.0000,11.000000,77.250000
Brandenburg,187.0000,0.000000,90.0000,4.000000,93.666667
Bremen,66.0000,0.000000,45.0000,1.000000,37.333333
Hamburg,66.0000,0.000000,79.0000,0.000000,72.500000
Hessen,387.0000,2.000000,364.0000,31.000000,196.000000
Mecklenburg-Vorpommern,160.0000,3.000000,69.0000,13.000000,61.250000
Niedersachsen,483.0000,0.000000,889.0000,30.000000,467.333333


In [36]:
county_ownership_kinds = df.groupby(['ags', 'landkreis', 'traeger_art']).size().unstack().fillna(0)

In [37]:
qgrid.show_grid(county_ownership_kinds)

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


## How does that correlate to be a patient in these nursing homes? (as in: are private nursing homes cheaper?)

In [38]:
price_cats = [c for c in df.columns if 'Gesamtpreis' in c and not 'Ratio' in c]
ownership_kind_prices = df.groupby('traeger_art')[price_cats].mean()
ownership_kind_prices

,Kurzzeitpflege Allgemein Härtefall Gesamtpreis,Kurzzeitpflege Allgemein Pflegestufe 1 Gesamtpreis,Kurzzeitpflege Allgemein Pflegestufe 2 Gesamtpreis,Kurzzeitpflege Allgemein Pflegestufe 3 Gesamtpreis,Kurzzeitpflege Beatmungspflichtig Härtefall Gesamtpreis,Kurzzeitpflege Beatmungspflichtig Pflegestufe 1 Gesamtpreis,Kurzzeitpflege Beatmungspflichtig Pflegestufe 2 Gesamtpreis,Kurzzeitpflege Beatmungspflichtig Pflegestufe 3 Gesamtpreis,Kurzzeitpflege Demenzkrank Härtefall Gesamtpreis,Kurzzeitpflege Demenzkrank Pflegestufe 1 Gesamtpreis,Kurzzeitpflege Demenzkrank Pflegestufe 2 Gesamtpreis,Kurzzeitpflege Demenzkrank Pflegestufe 3 Gesamtpreis,Kurzzeitpflege Wachkoma Härtefall Gesamtpreis,Kurzzeitpflege Wachkoma Pflegestufe 1 Gesamtpreis,Kurzzeitpflege Wachkoma Pflegestufe 2 Gesamtpreis,Kurzzeitpflege Wachkoma Pflegestufe 3 Gesamtpreis,Nachtpflege Allgemein Pflegestufe 1 Gesamtpreis,Nachtpflege Allgemein Pflegestufe 2 Gesamtpreis,Nachtpflege Allgemein Pflegestufe 3 Gesamtpreis,Nachtpflege Demenzkrank Pflegestufe 1 Gesamtpreis,Nachtpflege Demenzkrank Pflegestufe 2 Gesamtpreis,Nachtpflege Demenzkrank Pflegestufe 3 Gesamtpreis,Tagespflege Allgemein Härtefall Gesamtpreis,Tagespflege Allgemein Pflegestufe 1 Gesamtpreis,Tagespflege Allgemein Pflegestufe 2 Gesamtpreis,Tagespflege Allgemein Pflegestufe 3 Gesamtpreis,Tagespflege Demenzkrank Pflegestufe 1 Gesamtpreis,Tagespflege Demenzkrank Pflegestufe 2 Gesamtpreis,Tagespflege Demenzkrank Pflegestufe 3 Gesamtpreis,Vollstationär Allgemein Härtefall Gesamtpreis,Vollstationär Allgemein Pflegestufe 1 Gesamtpreis,Vollstationär Allgemein Pflegestufe 2 Gesamtpreis,Vollstationär Allgemein Pflegestufe 3 Gesamtpreis,Vollstationär Beatmungspflichtig Härtefall Gesamtpreis,Vollstationär Beatmungspflichtig Pflegestufe 1 Gesamtpreis,Vollstationär Beatmungspflichtig Pflegestufe 2 Gesamtpreis,Vollstationär Beatmungspflichtig Pflegestufe 3 Gesamtpreis,Vollstationär Demenzkrank Härtefall Gesamtpreis,Vollstationär Demenzkrank Pflegestufe 1 Gesamtpreis,Vollstationär Demenzkrank Pflegestufe 2 Gesamtpreis,Vollstationär Demenzkrank Pflegestufe 3 Gesamtpreis,Vollstationär Wachkoma Härtefall Gesamtpreis,Vollstationär Wachkoma Pflegestufe 1 Gesamtpreis,Vollstationär Wachkoma Pflegestufe 2 Gesamtpreis,Vollstationär Wachkoma Pflegestufe 3 Gesamtpreis,Vollstationär Allgemein Pflegestufe 1 Gesamtpreis + Investitionskosten Minimum
traeger_art,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
freigemeinnützig,3351.036575,2313.195051,2817.995191,3360.302947,4295.300000,2498.724194,2572.900000,2643.104762,3435.000000,2210.126882,2279.458065,2359.392473,4555.10000,2284.818182,2337.814286,2395.648872,1407.895082,1598.749180,1887.516393,1234.5,1564.5,1918.5,2023.184810,1540.229837,1761.305967,2017.240145,1131.9375,1531.425,1893.6,3548.719305,2247.150890,2748.447595,3283.449076,3307.246875,2325.420000,2865.40000,3260.201351,3717.798425,2200.656213,2632.073964,2989.858465,3943.714545,2200.736207,2766.672193,3258.003175,2602.194660
nicht zu ermitteln,3768.161538,2307.085714,2861.003571,3440.646429,NaN,NaN,NaN,NaN,3433.200000,2049.900000,2551.500000,3055.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1613.325000,1917.600000,2269.875000,NaN,NaN,NaN,3795.573529,2291.967568,2844.210811,3435.924324,NaN,NaN,NaN,NaN,3921.900000,2372.640000,2958.480000,3547.140000,NaN,NaN,NaN,NaN,2669.651351
privat,2836.231812,2035.427839,2445.168834,2896.467097,4521.253846,2137.710345,2275.880899,2396.952809,3128.788889,2051.373859,2132.030290,2220.418257,4412.26875,2081.794521,2198.307383,2289.789262,1335.157895,1488.260526,1726.350000,NaN,NaN,NaN,1812.259459,1327.279646,1540.368584,1760.273894,1197.6000,1447.200,1702.8,3119.147196,1946.260192,2369.606654,2810.842198,3560.420192,2268.554286,2812.90885,3236.636283,3501.982941,1981.273643,2389.123057,2728.878497,3837.104082,2071.250526,2598.194118,3062.829808,2372.304053
öffentlich,3411.787500,2307.902222,2789.357778,3319.575556,3863.100000,2898.300000,3698.900000,4126.100000,3442.050000,2539.269231,2744.33

In [39]:
# Average across all prices
ownership_kind_prices_mean = ownership_kind_prices.mean(1)
ownership_kind_prices_mean

traeger_art
freigemeinnützig      2546.048790
nicht zu ermitteln    2855.264242
privat                2420.110030
öffentlich            2956.158702
dtype: float64

## Where are people hungry, thirsty or hurt?

In [40]:
criteria = {
    'hurt': 'T05',
    'hungry': 'T08',
    'thirsty': 'T11'
}
for c in criteria.values():
    df[c + '_ratio'] = df[c] / df[c + '_total']

In [41]:
cost_1_total = 'Vollstationär Allgemein Pflegestufe 1 Gesamtpreis'
cost_invest_min = 'Vollstationär Allgemein Investitionskosten Minimum'
cost_1_total_invest_min = cost_1_total + ' + Investitionskosten Minimum'

c = ['Name', 'PLZ', 'Ort', 'traeger_art', 'traeger_name', 'Vollstationär Allgemein Pflegestufe 1 Gesamtpreis',
     'Vollstationär Allgemein Pflegestufe 2 Gesamtpreis',  'Vollstationär Allgemein Pflegestufe 3 Gesamtpreis',
    'Vollstationär Allgemein Investitionskosten Minimum', 'Vollstationär Allgemein Investitionskosten Maximum', cost_1_total_invest_min]
df[cost_1_total_invest_min] = df[cost_1_total] + df[cost_invest_min]
df[df[cost_1_total_invest_min] > 60].sort_values(cost_1_total_invest_min)[c].head(30).to_csv('export/billige_heime_investitionskosten.csv')

In [42]:
df[df['Name'].str.contains('Bad Honnef')][c + ['Telefon']]

,Name,PLZ,Ort,traeger_art,traeger_name,Vollstationär Allgemein Pflegestufe 1 Gesamtpreis,Vollstationär Allgemein Pflegestufe 2 Gesamtpreis,Vollstationär Allgemein Pflegestufe 3 Gesamtpreis,Vollstationär Allgemein Investitionskosten Minimum,Vollstationär Allgemein Investitionskosten Maximum,Vollstationär Allgemein Pflegestufe 1 Gesamtpreis + Investitionskosten Minimum,Telefon
9150,Parkresidenz Bad Honnef,53604,Bad Honnef,privat,"Parkresidenz Bad Honnef GmbH, Hauptstr. 450, 5...",860.1,860.1,860.1,360.6,360.6,1220.7,02224/1830


## Hurt

In [43]:
c = ['Name', 'landkreis', 'bundesland']
print(df['T05_ratio'].mean())
df.groupby('bundesland')['T05_ratio'].agg([np.mean, len])

0.81275177305


,mean,len
bundesland,,
Baden-Württemberg,0.966132,1887.0
Bayern,0.772025,1848.0
Berlin,0.952756,397.0
Brandenburg,0.797935,532.0
Bremen,0.632576,156.0
Hamburg,0.538365,241.0
Hessen,0.835093,1040.0
Mecklenburg-Vorpommern,0.804455,405.0
Niedersachsen,0.760863,1951.0


In [44]:
c = ['Name', 'Ort', 'Web', 'belegt_vollstationär', 'grade_overall', 'grade_care',
     'personal_total', 'personal_pflege', 'traeger_name', 'traeger_art', 'traeger_verband', 'start_date',
     'Vollstationär Allgemein Pflegestufe 1 Gesamtpreis', 'Vollstationär Allgemein Pflegestufe 2 Gesamtpreis',
     'Vollstationär Allgemein Pflegestufe 3 Gesamtpreis', 'Vollstationär Allgemein Investitionskosten Minimum',
     'Vollstationär Allgemein Investitionskosten Maximum', cost_1_total_invest_min]

cities = {
    'koeln': ['05315']
}

city_data = {}

for k, v in cities.items():
    print('koeln')
    city_data[k] = df[df['ags'].isin(v)][c]
    city_data[k].to_csv('export/data_%s.csv' % k, index=False, encoding='utf-8')


koeln


## Köln

In [45]:
city_data['koeln']

,Name,Ort,Web,belegt_vollstationär,grade_overall,grade_care,personal_total,personal_pflege,traeger_name,traeger_art,traeger_verband,start_date,Vollstationär Allgemein Pflegestufe 1 Gesamtpreis,Vollstationär Allgemein Pflegestufe 2 Gesamtpreis,Vollstationär Allgemein Pflegestufe 3 Gesamtpreis,Vollstationär Allgemein Investitionskosten Minimum,Vollstationär Allgemein Investitionskosten Maximum,Vollstationär Allgemein Pflegestufe 1 Gesamtpreis + Investitionskosten Minimum
8604,Seniorenhaus St. Maria,Köln,http://www.sh-st-maria.de,96.0,1.0,1.0,NaN,52.0,Stiftung der Cellitinnen zur heiligen Maria,freigemeinnützig,Diözesan-Caritasverband für das Erzbistum Köln...,1990-04-15,2745.3,3423.3,4126.8,360.6,360.6,3105.9
8605,Residenz am Dom,Köln,NaN,90.0,1.0,1.1,NaN,NaN,Residenz am Dom gGmbH,freigemeinnützig,Deutscher Paritätischer Wohlfahrtsverband - Ge...,2000-09-01,2112.0,2649.3,3206.1,360.6,360.6,2472.6
8606,St. Vincenz-Haus,Köln,NaN,128.0,1.0,1.0,NaN,NaN,"Stiftung St. Vincenz-Haus gGmbH, Konrad-Adenau...",freigemeinnützig,Diözesan-Caritasverband für das Erzbistum Köln...,1985-01-01,2930.7,3712.5,4523.1,360.6,360.6,3291.3
8607,CBT-Wohnhaus An St. Georg,Köln,http://www.cbt-gmbh.de,76.0,1.0,1.1,NaN,51.0,CBT-GmbH,freigemeinnützig,Diözesan-Caritasverband für das Erzbistum Köln...,1979-01-01,2676.0,3335.1,4018.5,360.6,360.6,3036.6
8608,Pflegeeinrichtung Clara-Elisen-Stift,Köln,NaN,83.0,1.5,1.9,NaN,NaN,Clara Elisen Stift zu Köln,freigemeinnützig,Diakonisches Werk der evangelischen Kirche im ...,2010-07-14,2569.2,3219.9,3895.5,360.6,360.6,2929.8
8609,Seniorenzentrum Arnold-Overzier-Haus,Köln,http://www.awo-arnold-overzier-haus.de,176.0,1.0,1.0,NaN,NaN,AWO Kreisverband Köln e. V.,freigemeinnützig,AWO Bezirksverband Mittelrhein e. V.,1984-04-01,2575.5,3211.8,3872.1,360.6,360.6,2936.1
8610,Seniorenzentrum Herz Jesu,Köln,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2655.9,3330.6,4029.6,360.6,360.6,3016.5
8611,Seniorenzentrum St. Josefshaus,Köln,NaN,33.0,1.0,1.0,NaN,NaN,"Marienhaus Seniorendienste GmbH, Margaretha-Fl...",freigemeinnützig,Diözesan-Caritasverband für das Erzbistum Köln...,2007-07-01,2757.0,3432.3,4133.4,360.6,360.6,3117.6
8612,Caritas-Altenzentrum St. Heribert,Köln,http://caritas.erzbistum-koeln.de/koeln_cv/pfl...,108.0,1.0,1.1,NaN,NaN,Caritasverband für die Stadt Köln e. V.,freigemeinnützig,Diözesan-Caritasverband für das Erzbistum Köln...,2005-04-05,2835.6,3576.3,4344.0,360.6,360.6,3196.2
8613,Hausgemeinschaften St. Augustinus,Köln,http://www.sh-st-augustinus.de,59.0,1.0,1.0,NaN,52.0,Seniorenhaus GmbH der Cellitinnen zur Hl. Gert...,freigemeinnützig,Diözesan-Caritasverband für das Erzbistum Köln...,2006-02-01,3457.8,3791.4,4125.0,360.6,360.6,3818.4


### What are the cheapest homes? Private, non-profit or municipal?

In [46]:
city_data['koeln'].groupby('traeger_art')['Vollstationär Allgemein Pflegestufe 1 Gesamtpreis'].agg([np.mean, len])

,mean,len
traeger_art,,
freigemeinnützig,2665.918182,77.0
nicht zu ermitteln,2883.000000,1.0
privat,2313.112500,8.0
öffentlich,3508.350000,8.0


### What are the best homes as for medical grade? Private, non-profit or municipal?

In [47]:
city_data['koeln'].groupby('traeger_art')['grade_care'].agg([np.mean, len])

,mean,len
traeger_art,,
freigemeinnützig,1.246753,77.0
nicht zu ermitteln,1.000000,1.0
privat,1.237500,8.0
öffentlich,1.087500,8.0


### Which insurance company is responsible for how many homes? 

In [48]:
city_data['koeln']['traeger_name'].value_counts().to_frame().head(10)

,traeger_name
"SBK Sozial-Betriebe-Köln gGmbH, Boltensternstr. 16, 50735 Köln",10
"Seniorenhaus GmbH der Cellitinnen zur Hl. Gertrud, Graseggerstr. 105, 50737 Köln",4
Caritasverband für die Stadt Köln e. V.,4
Clarenbachwerk Köln gGmbH,4
"Caritasverband für die Stadt Köln e.V., Bartholomäus-Schink-Straße 6, 50825 Köln",2
"Alexianer Köln GmbH, Kölner Str. 64, 51149 Köln",2
Alexianer Köln gGmbH,2
"Johanniter Seniorenhäuser GmbH, Niederlassung Regionalzentrum West, Siegburger Str. 197, 50679 Köln",2
Marienborn gGmbH Gesellschaft der Cellitinnen,2
AWO Kreisverband Köln e. V.,1


In [49]:
import folium

# We use the geo json file for our metro we generated above
# geo_path = r'kreise.geojson'
geo_path = 'geodata/kreise.topo.json'

def make_map(geo_data, key, filename='map_%s.html', scheme='YlOrRd', scale=None, legend_name=''):
    if scale is None:
        min_val = geo_data[key].min()
        q0 = geo_data[key].quantile( .05)
        q1 = geo_data[key].quantile( .25)
        q2 = geo_data[key].quantile( .5)
        q3 = geo_data[key].quantile( .75)
        max_val = geo_data[key].max()

        scale = [min_val - 1, q0, q1, q2, q3, max_val + 1]

    print(scale)
    m = folium.Map(location=[52, 10], zoom_start=6, height=800)
    m.geo_json(geo_path=geo_path, data=geo_data, data_out='%s_map.json' % key,
                 columns=['ags', key],
                 threshold_scale=scale,
                 key_on='feature.properties.AGS',
                 fill_color=scheme, fill_opacity=0.7, line_opacity=0.3,
                 legend_name=legend_name,
                 topojson='objects.kreise'
    )
    m.create_map(path=filename % key)
    return m


In [50]:
key = 'grade_care'
geo_data = df.groupby(['ags'])[[key]].mean().reset_index()
make_map(geo_data, key, legend_name='Median Care grade')

[0.10909090909090913, 1.2290700068634179, 1.3648809523809524, 1.5361904761904759, 1.7097727272727274, 3.3444444444444446]


In [51]:
df.pivot_table(index=['ags', 'landkreis'], values=['grade_overall'], aggfunc=np.mean).sort_values('grade_overall', ascending=False)

,,grade_overall
ags,landkreis,
07340,Südwestpfalz,1.700000
07135,Cochem-Zell,1.669231
07141,Rhein-Lahn-Kreis,1.664286
09473,Coburg,1.658333
09463,Coburg,1.650000
07137,Mayen-Koblenz,1.645455
07332,Bad Dürkheim,1.621429
09577,Weißenburg-Gunzenhausen,1.612500
03405,Wilhelmshaven,1.590909


In [52]:
c = ['Name', 'Ort', 'Web', 'belegt_vollstationär', 'grade_overall', 'grade_care',
     'personal_total', 'personal_pflege', 'traeger_name', 'traeger_art', 'traeger_verband', 'start_date',
     'Vollstationär Allgemein Pflegestufe 1 Gesamtpreis', 'Vollstationär Allgemein Pflegestufe 2 Gesamtpreis',
     'Vollstationär Allgemein Pflegestufe 3 Gesamtpreis', 'Vollstationär Allgemein Investitionskosten Minimum',
     'Vollstationär Allgemein Investitionskosten Maximum']
laender = df['bundesland'].value_counts()
laender = list(laender.index)
for land in laender:
    row = df[df['bundesland'] == land][c].sort_values('grade_care', ascending=False).iloc[0]
    print('%s: %s in %s hat medizinische Note %s - URL: %s' % (land, row['Name'], row['Ort'], row['grade_care'], row['Web']))

Nordrhein-Westfalen: AWO-Seniorenzentrum Erndtebrück in Erndtebrück hat medizinische Note 5.0 - URL: nan
Niedersachsen: Haus St. Marien in Georgsmarienhütte hat medizinische Note 4.6 - URL: nan
Baden-Württemberg: Priv. Pflegeheim Gerstetten Inh. Renate Filpe in Gerstetten hat medizinische Note 4.2 - URL: nan
Bayern: Lebenshilfe e. V. in Teisendorf hat medizinische Note 5.0 - URL: nan
Hessen: Pflege- & Therapie-Zentrum Inh. Bernd Volpp in Darmstadt hat medizinische Note 4.7 - URL: nan
Sachsen: Aventinum Seniorenwohnpark Hartha GmbH Kurzzeitpflege in Kurort Hartha hat medizinische Note 5.0 - URL: nan
Schleswig-Holstein: Alten- und Pflegeheim Haus Diana in Seedorf hat medizinische Note 4.4 - URL: nan
Sachsen-Anhalt: Pflegezentrum Großkugel, Haus Abendfrieden in Kabelsketal OT Großkugel hat medizinische Note 4.3 - URL: nan
Brandenburg: Seniorenzentrum Caputh in Schwielowsee hat medizinische Note 2.9 - URL: http://www.edbtl.de
Rheinland-Pfalz: AWO Seniorenzentrum Altes Brauhaus zur Nette un

In [54]:
(df.groupby(['lat', 'lng']).size().sort_values(ascending=False) > 1).sum()

1210

In [55]:
df.groupby(['Name', 'PLZ']).size().sort_values(ascending=False)

Name                                                                PLZ  
Tagespflege Jedermann                                               14776    3
Tagesbetreuungsstätte Wardenburg                                    26203    2
Altenheim Siebenbürgen                                              37520    2
Seniorenzentrum "Negendanksland" Beelitz                            14547    2
Engelsstift                                                         51588    2
Wohn- und Pflegezentrum Lindenhof GmbH                              37269    2
Mediana Seniorenresidenz GmbH                                       36088    2
Betreuungshaus Block & Wagner                                       51597    2
Caritashaus St. Elisabeth                                           41569    2
KLEEBLATT PFLEGEHEIM ASPERG                                         71679    2
AWO-Seniorenzentrum Christian-Dierig-Haus                           86157    2
Wohn- und Ferienhaus e.V.                                

In [56]:
from slugify import slugify

df['slug_raw'] = df.apply(lambda x: '%s-%s' % (x['PLZ'], slugify(x['Name'])), 1)


In [57]:
import functools
stationary_columns = [c for c in list(df.columns) if c.startswith(('Vollstationär', 'Kurzzeitpflege'))]
stationary_filter = functools.reduce(lambda x, y: x | y, (df[c].notnull() for c in stationary_columns))
df_stationary = df[stationary_filter]
len(df_stationary)

11869

In [58]:
df['slug'] = df['slug_raw'].copy()
groups = df.groupby('slug_raw')
for key, group_df in groups:
    if len(group_df) < 2:
        continue
    for i, (index, row) in enumerate(group_df.iterrows()):
        if i == 0:
            continue
        df.set_value(index, 'slug', '%s-%s' % (row['slug_raw'], i))


In [59]:
df[df['slug_raw'].duplicated()][['Name', 'slug', 'slug_raw', 'Ort', 'PLZ']].head()

,Name,slug,slug_raw,Ort,PLZ
95,Altenpflegeheim Neufriedstein,01445-altenpflegeheim-neufriedstein-1,01445-altenpflegeheim-neufriedstein,Radebeul,01445
764,K & S Seniorenresidenz Torgau - Haus Renaissance,04860-k-s-seniorenresidenz-torgau-haus-renaiss...,04860-k-s-seniorenresidenz-torgau-haus-renaiss...,Torgau,04860
1166,Pflegeheim der AWO,07318-pflegeheim-der-awo-1,07318-pflegeheim-der-awo,Saalfeld,07318
2102,"Seniorenzentrum ""Negendanksland"" Beelitz",14547-seniorenzentrum-negendanksland-beelitz-1,14547-seniorenzentrum-negendanksland-beelitz,Beelitz,14547
2157,Tagespflege Jedermann,14776-tagespflege-jedermann-1,14776-tagespflege-jedermann,Brandenburg,14776
2158,Tagespflege Jedermann,14776-tagespflege-jedermann-2,14776-tagespflege-jedermann,Brandenburg,14776
2401,Stephanus Wohnen und Pflege Seniorenzentrum El...,16562-stephanus-wohnen-und-pflege-seniorenzent...,16562-stephanus-wohnen-und-pflege-seniorenzent...,Hohen Neuendorf,16562
3007,Pflegezentrum Hainfelder Hof,21435-pflegezentrum-hainfelder-hof-1,21435-pflegezentrum-hainfelder-hof,Stelle,21435
3855,Cecilien-Burg Privat-Alten-Pflegeheim,25436-cecilien-burg-privat-alten-pflegeheim-1,25436-cecilien-burg-privat-alten-pflegeheim,Tornesch,25436
3914,Ev. Altenhilfezentrum Meldorf,25704-ev-altenhilfezentrum-meldorf-1,25704-ev-altenhilfezentrum-meldorf,Meldorf,25704


In [60]:
df.to_csv('data/nursinghomes_import.csv', encoding='utf-8', index=False)

## Some aggregations

In [61]:
df['grade_care'].mean()

1.5039863223252046

In [62]:
df['grade_overall'].mean()

1.277251867182579

In [63]:
(df['Vollstationär Allgemein Härtefall Gesamtpreis'] + df['Vollstationär Allgemein Investitionskosten Minimum']).mean()

3749.8115336967294

In [64]:
df.groupby('bundesland')['Vollstationär Allgemein Pflegestufe 3 Gesamtpreis'].mean().sort_values()

bundesland
Sachsen-Anhalt            2449.654585
Mecklenburg-Vorpommern    2570.542979
Sachsen                   2602.469255
Niedersachsen             2672.730908
Thüringen                 2701.908738
Schleswig-Holstein        2704.608088
Brandenburg               2818.216399
Bayern                    3040.587038
Hessen                    3077.930552
Bremen                    3081.322340
Berlin                    3201.488542
Hamburg                   3377.285714
Baden-Württemberg         3384.198515
Rheinland-Pfalz           3453.237885
Nordrhein-Westfalen       3601.165062
Saarland                  3614.259060
Name: Vollstationär Allgemein Pflegestufe 3 Gesamtpreis, dtype: float64

In [65]:
(df['Vollstationär Allgemein Härtefall Gesamtpreis'] + df['Vollstationär Allgemein Investitionskosten Maximum']).mean()

3811.4096603773583

In [66]:
df['belegt_vollstationär'].mean()

69.594708899487088

In [67]:
df['days_in_operation'].mean() / 365

21.116388568301218

In [75]:
(df.groupby('bundesland')['Vollstationär Allgemein Pflegestufe 3 Gesamtpreis'].mean()).sort_values()

bundesland
Sachsen-Anhalt            2449.654585
Mecklenburg-Vorpommern    2570.542979
Sachsen                   2602.469255
Niedersachsen             2672.730908
Thüringen                 2701.908738
Schleswig-Holstein        2704.608088
Brandenburg               2818.216399
Bayern                    3040.587038
Hessen                    3077.930552
Bremen                    3081.322340
Berlin                    3201.488542
Hamburg                   3377.285714
Baden-Württemberg         3384.198515
Rheinland-Pfalz           3453.237885
Nordrhein-Westfalen       3601.165062
Saarland                  3614.259060
Name: Vollstationär Allgemein Pflegestufe 3 Gesamtpreis, dtype: float64

In [76]:
df[df['Ort'] == 'Berlin'].to_csv('data/nursinghomes_berlin.csv', encoding='utf-8', index=False)

In [77]:
len(df.landkreis.value_counts(dropna=False))

381

In [78]:
len(df), len(df.lat.value_counts(dropna=False))

(14935, 12261)

In [79]:
len(df.landkreis.dropna())

14928

In [82]:
df[df.bundesland == 'Berlin'].grade_care.mean()

1.2647249190938512

In [83]:
df.traeger_art.value_counts()

freigemeinnützig      6135
privat                4509
öffentlich             432
nicht zu ermitteln      37
Name: traeger_art, dtype: int64

In [84]:
df[u'Vollstationär Allgemein Pflegestufe 3 Eigenanteil'].mean()

1869.0822418021617

In [85]:
k = u'Vollstationär Allgemein Pflegestufe 3 Eigenanteil'
df.dropna(subset=[k]).groupby('bundesland')[k].quantile([0.2, 0.4, 0.6, 0.8]).unstack()

,0.2,0.4,0.6,0.8
bundesland,,,,
Baden-Württemberg,1741.70,2129.78,2350.04,2526.62
Bayern,1505.00,1680.80,1838.54,2008.22
Berlin,1720.40,1885.34,2003.12,2094.68
Brandenburg,1337.00,1449.50,1524.50,1641.80
Bremen,1693.70,1803.92,1955.36,2097.32
Hamburg,1879.88,2034.50,2208.44,2355.80
Hessen,1628.90,1845.20,2036.90,2223.20
Mecklenburg-Vorpommern,1097.42,1216.88,1317.20,1450.16
Niedersachsen,1274.24,1430.24,1573.52,1750.04


In [86]:
k = u'belegt_vollstationär'
df.dropna(subset=[k]).groupby('bundesland')[k].quantile([0.2, 0.4, 0.6, 0.8]).unstack()

,0.2,0.4,0.6,0.8
bundesland,,,,
Baden-Württemberg,29.0,45.0,64.0,91.0
Bayern,38.0,61.0,83.0,108.0
Berlin,42.6,75.2,105.0,139.0
Brandenburg,40.0,57.0,74.0,102.0
Bremen,19.8,46.0,65.0,82.0
Hamburg,48.6,82.6,108.0,142.8
Hessen,30.0,48.0,71.0,99.0
Mecklenburg-Vorpommern,46.0,62.0,77.4,105.2
Niedersachsen,33.0,51.0,67.0,91.0


In [87]:
k = u'days_in_operation'
df.dropna(subset=[k])[k].quantile([0.2, 0.4, 0.6, 0.8]) / 365

0.2     8.090411
0.4    13.339726
0.6    19.432329
0.8    31.704110
Name: days_in_operation, dtype: float64

In [88]:
df.groupby('bundesland')['grade_care'].mean()

bundesland
Baden-Württemberg         1.362950
Bayern                    1.647086
Berlin                    1.264725
Brandenburg               1.288612
Bremen                    1.848214
Hamburg                   1.688966
Hessen                    1.478189
Mecklenburg-Vorpommern    1.398776
Niedersachsen             1.574394
Nordrhein-Westfalen       1.462506
Rheinland-Pfalz           1.822955
Saarland                  1.256738
Sachsen                   1.341103
Sachsen-Anhalt            1.393562
Schleswig-Holstein        1.666387
Thüringen                 1.688953
Name: grade_care, dtype: float64